In [65]:
import os
os.chdir('..')
from inputGetter import get_input
from collections import defaultdict

content = get_input(2024, 12)[:-1]

In [66]:
content = [list(line) for line in content.split('\n')]

sz_r, sz_c = len(content), len(content[0])
directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]

def dfs(cells_by_rows, r, c):
    area, perimeter = 1, 4
    cells_by_rows[r].append(c)
    ch = content[r][c]
    content[r][c] = '.'
    
    for add_r, add_c in directions:
        new_r, new_c = r + add_r, c + add_c
        if 0 <= new_r < sz_r and 0 <= new_c < sz_c and content[new_r][new_c] == ch:
            perimeter -= 1
            if content[new_r][new_c] != '.':
                next_area, next_perimeter = dfs(cells_by_rows, new_r, new_c)
                area += next_area
                perimeter += next_perimeter
    
    return area, perimeter

In [67]:
res_a, res_b = 0, 0
for r in range(sz_r):
    for c in range(sz_c):
        if content[r][c] != '.':
            cells_by_rows = defaultdict(list)
            area, perimeter = dfs(cells_by_rows, r, c)
            
            # This contains rectangles in each row. Created by grouping consecutive numbers into groups
            # E.g.: [1, 2, 3, 4, 7, 8, 9] -> [(1, 4), (7, 9)]
            rectangles_by_rows = defaultdict(list)
            for row, cols in cells_by_rows.items():
                cols.sort()
                start = end = cols[0]
                for col in cols[1:]:
                    if col > end + 1:
                        rectangles_by_rows[row].append((start, end))
                        start = col
                    end = col
                        
                rectangles_by_rows[row].append((start, end))
        
            sides = 0
            for row, rectangles in list(rectangles_by_rows.items()):
                for start_c, end_c in rectangles:
                    sides += 4  # Normally each rectangle should has 4 sides, but will be fixed based on checks below
                    
                    # Checking top rectangles of the current rectangle, which will affect the top side
                    for top_start, top_end in rectangles_by_rows[row-1]:
                        if top_start > end_c:
                            break
                        
                        # Remove left or right side of this rectangle, because it's just continue the existing left/right side from the top rectangle
                        if top_start == start_c:
                            sides -= 1
                        if top_end == end_c:
                            sides -= 1
                        
                        # Remove top side of this rectangle, as it's covered by the top rectangle entirely
                        if top_start <= start_c and top_end >= end_c:
                            sides -= 1
                            break
                        # Adding to the top, as the top rectangle stays in between, and split the top side into 2 sides
                        if top_start > start_c and top_end < end_c:
                            sides += 1
                    
                    # Checking bottom rectangles of the current rectangle, which will affect the bottom side
                    for bottom_start, bottom_end in rectangles_by_rows[row+1]:
                        if bottom_start > end_c:
                            break
                        
                        # Remove bottom side of this rectangle, as it's covered by the bottom rectangle entirely
                        if bottom_start <= start_c and bottom_end >= end_c:
                            sides -= 1
                            break
                        # Adding to the bottom, as the bottom rectangle stays in between, and split the top side into 2 sides
                        if bottom_start > start_c and bottom_end < end_c:
                            sides += 1
        
            res_a += perimeter * area
            res_b += sides * area

print(f'Part A: {res_a}')
print(f'Part B: {res_b}')

Part A: 6400750
Part B: 901100
